In [1]:
import os
from datetime import datetime
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
import xarray as xr
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas
import json
import panel as pn
import geoviews
import earthaccess
from rasterio.enums import ColorInterp
from rasterio.mask import mask
from rasterio.warp import Resampling
from shapely.geometry import Polygon
from shapely.ops import transform
from tqdm.auto import tqdm
import cupy as cp
import time
import nasa_hls

In [2]:
# There are a couple plots that generate errors that we want to ignore.
import warnings
warnings.filterwarnings('ignore')

In [3]:
earthaccess.login(persist=True)

In [4]:
field = gp.read_file('G:/2024_활동지도/행정경계/korea.shp')
field

,dissolve,geometry
0,1.0,"MULTIPOLYGON (((126.26475 33.11572, 126.26476 ..."


In [5]:
grid = gp.read_file('G:/grid/fin/S2A_OPER_GRID_utm.shp')
grid = grid[['Name', 'geometry']]
tile = grid['Name']
granule_names = list(tile)

In [6]:
bbox = tuple(list(field.total_bounds))
bbox

(124.60971767857728, 33.1123557596338, 131.87278314808853, 38.61370930257595)

In [7]:
year = '2024'

In [8]:
temporal = (f"{year}-05-01T00:00:00", f"{year}-06-15T23:59:59")

In [9]:
earthaccess.search_data

<function earthaccess.api.search_data(count: int = -1, **kwargs: typing_extensions.Any) -> List[earthaccess.results.DataGranule]>

In [10]:
results = []

for name in tqdm(granule_names):
    result = earthaccess.search_data(
        short_name=['HLSL30', 'HLSS30'],
        bounding_box=bbox,
        temporal=temporal,
        count=100,
        cloud_cover=(0, 50),
        granule_name=f'*{name}*'  # 와일드카드로 granule_name에 해당하는 패턴 검색
    )
    print(f"Granule '{name}' - 검색된 개수: {len(result)}")
    results.extend(result)  # 결과를 results 리스트에 추가

  0%|          | 0/36 [00:00<?, ?it/s]

Granule '51SXB' - 검색된 개수: 8
Granule '51SXC' - 검색된 개수: 9
Granule '51SXT' - 검색된 개수: 12
Granule '51SYA' - 검색된 개수: 15
Granule '51SYB' - 검색된 개수: 14
Granule '51SYC' - 검색된 개수: 19
Granule '51SYS' - 검색된 개수: 13
Granule '51SYT' - 검색된 개수: 13
Granule '51SYU' - 검색된 개수: 18
Granule '51SYV' - 검색된 개수: 12
Granule '52SBB' - 검색된 개수: 14
Granule '52SBC' - 검색된 개수: 7
Granule '52SBD' - 검색된 개수: 10
Granule '52SBE' - 검색된 개수: 15
Granule '52SBF' - 검색된 개수: 16
Granule '52SBG' - 검색된 개수: 13
Granule '52SBH' - 검색된 개수: 19
Granule '52SCB' - 검색된 개수: 14
Granule '52SCC' - 검색된 개수: 14
Granule '52SCD' - 검색된 개수: 19
Granule '52SCE' - 검색된 개수: 18
Granule '52SCF' - 검색된 개수: 18
Granule '52SCG' - 검색된 개수: 14
Granule '52SCH' - 검색된 개수: 21
Granule '52SDD' - 검색된 개수: 15
Granule '52SDE' - 검색된 개수: 18
Granule '52SDF' - 검색된 개수: 18
Granule '52SDG' - 검색된 개수: 18
Granule '52SDH' - 검색된 개수: 24
Granule '52SED' - 검색된 개수: 14
Granule '52SEE' - 검색된 개수: 16
Granule '52SEF' - 검색된 개수: 18
Granule '52SEG' - 검색된 개수: 22
Granule '52SFG' - 검색된 개수: 16
Granule '52SGG' -

In [11]:
pd.json_normalize(results).head(5)

,size,meta.concept-type,meta.concept-id,meta.revision-id,meta.native-id,meta.collection-concept-id,meta.provider-id,meta.format,meta.revision-date,umm.TemporalExtent.RangeDateTime.BeginningDateTime,...,umm.CollectionReference.EntryTitle,umm.RelatedUrls,umm.DataGranule.DayNightFlag,umm.DataGranule.Identifiers,umm.DataGranule.ProductionDateTime,umm.DataGranule.ArchiveAndDistributionInformation,umm.Platforms,umm.MetadataSpecification.URL,umm.MetadataSpecification.Name,umm.MetadataSpecification.Version
0,157.224716,granule,G2991239576-LPCLOUD,1,HLS.L30.T51SXB.2024129T021650.v2.0,C2021957657-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-10T07:24:06.306Z,2024-05-08T02:16:50.544Z,...,HLS Landsat Operational Land Imager Surface Re...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.L30.T51SXB.2024129T021650...,2024-05-10T07:19:37.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 16486...","[{'ShortName': 'LANDSAT-9', 'Instruments': [{'...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
1,154.965186,granule,G2995057181-LPCLOUD,1,HLS.S30.T51SXB.2024130T022551.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-11T07:32:05.234Z,2024-05-09T02:37:19.606Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T51SXB.2024130T022551...,2024-05-11T07:27:07.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 16249...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
2,160.627707,granule,G3008961922-LPCLOUD,1,HLS.S30.T51SXB.2024135T022529.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-16T08:58:01.379Z,2024-05-14T02:37:16.358Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T51SXB.2024135T022529...,2024-05-16T08:55:08.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 16843...","[{'ShortName': 'Sentinel-2B', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
3,128.251033,granule,G3014314338-LPCLOUD,1,HLS.L30.T51SXB.2024137T021621.v2.0,C2021957657-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-18T07:23:06.795Z,2024-05-16T02:16:21.112Z,...,HLS Landsat Operational Land Imager Surface Re...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.L30.T51SXB.2024137T021621...,2024-05-18T07:19:14.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 13448...","[{'ShortName': 'LANDSAT-8', 'Instruments': [{'...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
4,176.168429,granule,G3022947928-LPCLOUD,1,HLS.S30.T51SXB.2024140T022531.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-21T13:37:14.768Z,2024-05-19T02:37:18.636Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T51SXB.2024140T022531...,2024-05-21T13:34:58.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 18472...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6


In [12]:
len(results)

549

In [13]:
hls_results_urls = [granule.data_links() for granule in results]

In [14]:
# 중첩된 리스트에서 'HLS.L30.T52SCG'를 포함하는 URL만 추출하는 코드
filtered_hls_results_urls = [[url for url in sublist if 'HLS.L30.T52SCG' in url] for sublist in hls_results_urls]

# 빈 리스트가 아닌 결과만 유지
filtered_hls_results_urls = [sublist for sublist in filtered_hls_results_urls if sublist]

In [15]:
# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')

In [16]:
f_value = [0,4,16,20,32,36,48,52,64,68,80,84,96,100,112,116,128,132,144,148,160,164,176,180,192,196,208,212,224,228,240,244]

In [17]:
# Define function to scale 
def scaling(band):
    scale_factor = band.attrs['scale_factor'] 
    band_out = band.copy()
    band_out.data = band.data*scale_factor
    band_out.attrs['scale_factor'] = 1
    return(band_out)

In [29]:
def calc_ndvi(red, nir):
    # Create NDVI xarray.DataArray that has the same coordinates and metadata
    ndvi = red.copy()
    
    # Calculate the NDVI
    ndvi_data = (nir.data - red.data) / (nir.data + red.data)
    
    # Replace the Red xarray.DataArray data with the new NDVI data
    ndvi.data = ndvi_data
    
    # Exclude inf values
    ndvi = xr.where(ndvi != np.inf, ndvi, np.nan, keep_attrs=True)
    
    # Apply -1 and 1 clipping (set values outside this range to NaN)
    ndvi = xr.where((ndvi >= -1) & (ndvi <= 1), ndvi, np.nan, keep_attrs=True)
    
    # Change the long_name in the attributes
    ndvi.attrs['long_name'] = 'NDVI'
    ndvi.attrs['scale_factor'] = 1
    
    return ndvi

In [30]:
def calc_ndbi(nir, swir1):
    # Create NDBI xarray.DataArray that has the same coordinates and metadata
    ndbi = nir.copy()
    
    # Calculate the NDBI
    ndbi_data = (swir1.data - nir.data) / (swir1.data + nir.data)
    
    # Replace the NIR xarray.DataArray data with the new NDBI data
    ndbi.data = ndbi_data
    
    # Exclude inf values
    ndbi = xr.where(ndbi != np.inf, ndbi, np.nan, keep_attrs=True)
    
    # Apply -1 and 1 clipping (set values outside this range to NaN)
    ndbi = xr.where((ndbi >= -1) & (ndbi <= 1), ndbi, np.nan, keep_attrs=True)
    
    # Change the long_name in the attributes
    ndbi.attrs['long_name'] = 'NDBI'
    ndbi.attrs['scale_factor'] = 1
    
    return ndbi

In [31]:
def calc_savi(red, nir, L=0.5):
    # Create SAVI xarray.DataArray that has the same coordinates and metadata
    savi = red.copy()
    
    # Calculate the SAVI
    savi_data = ((nir.data - red.data) / (nir.data + red.data + L)) * (1 + L)
    
    # Replace the Red xarray.DataArray data with the new SAVI data
    savi.data = savi_data
    
    # Exclude the inf values
    savi = xr.where(savi != np.inf, savi, np.nan, keep_attrs=True)
    
    # Apply -1 and 1 clipping (set values outside this range to NaN)
    savi = xr.where((savi >= -1) & (savi <= 1), savi, np.nan, keep_attrs=True)

    
    # Change the long_name in the attributes
    savi.attrs['long_name'] = 'SAVI'
    savi.attrs['scale_factor'] = 1
    
    return savi

In [33]:
def calc_bsi(red, nir, blue, swir1):
    # Create BSI xarray.DataArray that has the same coordinates and metadata
    bsi = red.copy()

    # Calculate the BSI
    bsi_data = ((swir1.data + red.data) - (nir.data + blue.data)) / ((swir1.data + red.data) + (nir.data + blue.data))
    
    # Replace the Red xarray.DataArray data with the new BSI data
    bsi.data = bsi_data
    
    # Exclude the inf values (and NaN values)
    bsi = xr.where(bsi != np.inf, bsi, np.nan, keep_attrs=True)
    
    # Apply conditions:
    # - If BSI < -9999 -> NaN
    # - If BSI > 32767 -> NaN
    bsi = xr.where((bsi >= -9999) & (bsi <= 32767), bsi, np.nan, keep_attrs=True)
    
    
    # Change the long_name in the attributes
    bsi.attrs['long_name'] = 'BSI'
    bsi.attrs['scale_factor'] = 1

    return bsi

In [34]:
def filter_fmask(fmask, f_values):
    # Filter based on the fmask values in f_values
    mask = np.isin(fmask.data, f_values)
    
    # Apply the mask to the fmask, keeping only the values that are in f_values
    filtered_fmask = xr.where(mask, fmask, np.nan, keep_attrs=True)
    filtered_fmask = filtered_fmask.astype('uint8')
    
    return filtered_fmask

In [35]:
import logging
import os
from rasterio.errors import RasterioIOError  # RasterioIOError 추가

In [ ]:
for j, h in enumerate(hls_results_urls):
    
    outName1 = h[0].split('/')[-1].split('v2.0')[0] +'v2.0_NDVI_cropped.tif'
    outName2 = h[0].split('/')[-1].split('v2.0')[0] +'v2.0_SAVI_cropped.tif'
    outName3 = h[0].split('/')[-1].split('v2.0')[0] +'v2.0_BSI_cropped.tif'
    outName4 = h[0].split('/')[-1].split('v2.0')[0] +'v2.0_NDBI_cropped.tif'

    band_links = []
    if h[0].split('/')[4] == 'HLSS30.020':
        bands = ['B02', 'B04', 'B8A', 'B11', 'Fmask']  # NIR, RED, BLUE, SWIR1 for S30
        
    else:
        bands = ['B02', 'B04', 'B05', 'B06', 'Fmask']  # NIR, RED, BLUE, SWIR1 for L30       
    
    for a in h:
        if any(b in a for b in bands):
            band_links.append(a)

    # 각 outName에 대한 파일 존재 여부 개별 확인
    if os.path.exists(f'G:/2024_활동지도/mask/{year}_indices/{outName1}'):
        print(f"{outName1} has already been processed and is available in this directory, moving to next file.")
        continue
    if os.path.exists(f'G:/2024_활동지도/mask/{year}_indices/{outName2}'):
        print(f"{outName2} has already been processed and is available in this directory, moving to next file.")
        continue
    if os.path.exists(f'G:/2024_활동지도/mask/{year}_indices/{outName3}'):
        print(f"{outName3} has already been processed and is available in this directory, moving to next file.")
        continue
    if os.path.exists(f'G:/2024_활동지도/mask/{year}_indices/{outName4}'):
        print(f"{outName4} has already been processed and is available in this directory, moving to next file.")
        continue

    # Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
    chunk_size = dict(band=1, x=1830, y=1830)
    max_retries = 10

    try:
        for e in band_links:
            print(e)
            for _i in range(max_retries):
                try:
                    if e.rsplit('.', 2)[-2] == bands[0]:  # Blue (B02 index)
                        blue = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                        blue.attrs['scale_factor'] = 0.0001
                    elif e.rsplit('.', 2)[-2] == bands[1]:  # Red (B04 index)
                        red = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                        red.attrs['scale_factor'] = 0.0001
                    elif e.rsplit('.', 2)[-2] == bands[2]:  # NIR (B05 index)
                        nir = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                        nir.attrs['scale_factor'] = 0.0001
                    elif e.rsplit('.', 2)[-2] == bands[3]:  # SWIR1 (B06 index)
                        swir1 = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                        swir1.attrs['scale_factor'] = 0.0001
                    elif e.rsplit('.', 2)[-2] == bands[4]:  # FMASK band
                        fmask = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                    break  # 성공적으로 로드되면 루프 탈출
                except Exception as ex:
                    print(f"vsi curl error: {ex}. Retrying...")
            else:
                print(f"Failed to process {e} after {max_retries} retries. Skipping this band.")
                continue  # 오류 발생 시 해당 타일을 건너뜁니다.

        # 만약 모든 밴드를 성공적으로 불러오지 못했으면, 다음 타일로 넘어감
        if 'blue' not in locals() or 'red' not in locals() or 'nir' not in locals() or 'swir1' not in locals() or 'fmask' not in locals():
            print(f"Skipping {outName1}, {outName2}, {outName3}, {outName4} due to missing bands.")
            continue

        fsUTM = field.to_crs(nir.spatial_ref.crs_wkt)

        fmask_filtered = filter_fmask(fmask, f_value)
        
        red = xr.where(fmask_filtered, red, np.nan, keep_attrs=True)
        blue = xr.where(fmask_filtered, blue, np.nan, keep_attrs=True)
        nir = xr.where(fmask_filtered, nir, np.nan, keep_attrs=True)
        swir1 = xr.where(fmask_filtered, swir1, np.nan, keep_attrs=True)

        
        # Crop to our ROI and apply scaling and masking
        red_cropped = red.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
        red_cropped_scaled = scaling(red_cropped)

        blue_cropped = blue.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
        blue_cropped_scaled = scaling(blue_cropped)

        nir_cropped = nir.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
        nir_cropped_scaled = scaling(nir_cropped)

        swir1_cropped = swir1.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
        swir1_cropped_scaled = scaling(swir1_cropped)

        # Apply fmask filter
        

        print('Cropped and Filtered')

        # Generate NDVI, SAVI, BSI, NDBI with fmask filter
        ndvi_cropped = calc_ndvi(red_cropped_scaled, nir_cropped_scaled)
        savi_cropped = calc_savi(red_cropped_scaled, nir_cropped_scaled, L=0.5)
        bsi_cropped = calc_bsi(red_cropped_scaled, nir_cropped_scaled, blue_cropped_scaled, swir1_cropped_scaled)
        ndbi_cropped = calc_ndbi(swir1_cropped_scaled, nir_cropped_scaled)

        print('All indices Calculated')

        # Remove any observations that are entirely fill value, 각각의 분광지수에 대해 출력
        if np.nansum(ndvi_cropped.data) == 0.0:
            print(f"NDVI for file {outName1} contains only fill values and will not be exported.")
            continue
        if np.nansum(savi_cropped.data) == 0.0:
            print(f"SAVI for file {outName2} contains only fill values and will not be exported.")
            continue
        if np.nansum(bsi_cropped.data) == 0.0:
            print(f"BSI for file {outName3} contains only fill values and will not be exported.")
            continue
        if np.nansum(ndbi_cropped.data) == 0.0:
            print(f"NDBI for file {outName4} contains only fill values and will not be exported.")
            continue

        # Export rasters
        ndvi_cropped.rio.to_raster(raster_path=f'G:/2024_활동지도/mask/{year}_indices/{outName1}', driver='COG')
        savi_cropped.rio.to_raster(raster_path=f'G:/2024_활동지도/mask/{year}_indices/{outName2}', driver='COG')
        bsi_cropped.rio.to_raster(raster_path=f'G:/2024_활동지도/mask/{year}_indices/{outName3}', driver='COG')
        ndbi_cropped.rio.to_raster(raster_path=f'G:/2024_활동지도/mask/{year}_indices/{outName4}', driver='COG')

        print(f"Processed file {j+1} of {len(hls_results_urls)}")

    except RasterioIOError as e:
        print(f"RasterioIOError: {e}. Skipping to next tile.")
        continue
    except Exception as ex:
        print(f"An unexpected error occurred: {ex}. Skipping to next tile.")
        continue

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T51SXB.2024129T021650.v2.0/HLS.L30.T51SXB.2024129T021650.v2.0.B04.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T51SXB.2024129T021650.v2.0/HLS.L30.T51SXB.2024129T021650.v2.0.Fmask.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T51SXB.2024129T021650.v2.0/HLS.L30.T51SXB.2024129T021650.v2.0.B02.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T51SXB.2024129T021650.v2.0/HLS.L30.T51SXB.2024129T021650.v2.0.B06.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T51SXB.2024129T021650.v2.0/HLS.L30.T51SXB.2024129T021650.v2.0.B05.tif
Cropped and Filtered
All indices Calculated
Processed file 1 of 549
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T51SXB.2024130T022551.v2.0/HLS.S30.T51SXB.2024130T022551.v2.0.Fmask.tif
https://data.lpdaac.ea